In [17]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [25]:
import os
import copy
import types
import random
import argparse
import datetime
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import utils
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers import *
from data import *

In [19]:
def lstm_embedding_model(hidden_1, hidden_2,\
                         num_layers=1, reshape=None):
    global input_shape
    model = Sequential()
    if reshape:
        model.add(Reshape(reshape, input_shape=input_shape))
        model.add(LSTM(hidden_1, return_sequences=True))
    else:
        model.add(LSTM(hidden_1, return_sequences=True, input_shape=input_shape))
    if num_layers == 2:
        model.add(LSTM(hidden_2, return_sequences=True, input_shape=input_shape))
    return model

def lstm_counting_model(model, counting_hidden_1, counting_dense_1,\
                        counting_dense_2, kernel_initializer='normal',\
                        optimizer=None, learning_rate=0.001, dropout=None):
    
    if optimizer == 'adam' or optimizer is None:
        adam = keras.optimizers.Adam(lr=learning_rate)
    
    model.add(Masking(mask_value=0.0, name='mask'))
    model.add(LSTM(counting_hidden_1, return_sequences=False, name='counting_lstm_1'))
    model.add(Dense(counting_dense_1, activation='relu', kernel_initializer=kernel_initializer, name='counting_dense_1'))
    model.add(Dense(counting_dense_2, activation='relu', kernel_initializer=kernel_initializer, name='counting_dense_2'))
    model.add(Dense(1, kernel_initializer=kernel_initializer, name='output'))
    model.add(Activation('softplus'))
    model.compile(loss='poisson', optimizer=adam, metrics=['mae'])
    return model

def build_lstm_time_model(hidden_1, hidden_2, counting_hidden_1,\
                          counting_dense_1, counting_dense_2,\
                          kernel_initializer='normal',\
                          learning_rate=1e-2, optimizer='adam', dropout=None):
    
    model = lstm_embedding_model(hidden_1, hidden_2, reshape=(-1, 2))
    counting_model = lstm_counting_model(model, counting_hidden_1,\
                                         counting_dense_1, counting_dense_2,\
                                         kernel_initializer=kernel_initializer,\
                                         optimizer=optimizer, learning_rate=learning_rate, dropout=dropout)
    return counting_model

In [20]:
if True:
    data_type = 'time'
    window = 256
    loss = 'poisson'
    model_type = 'lstm_time'
    base_path = '/scratch/sk7898/pedbike/window_256'
    batch_size = 64

if data_type == 'stft':
    fileloc = os.path.join(base_path, 'downstream_stft')
elif data_type == 'time':
    fileloc = os.path.join(base_path, 'downstream_time')
else:
    raise ValueError('Only stft/time are valid data types')
    
x_train, x_val, x_test, y_train, y_val, y_test, seqs_train, seqs_val, seqs_test = get_data(fileloc)
n_bins = int(len(seqs_train)/batch_size)
    
assert x_train.shape[0] == y_train.shape[0] == seqs_train.shape[0]    

n_timesteps, n_features = None, window*2
input_shape=(n_timesteps, n_features)

train_gen = train_generator(n_bins, x_train, y_train, seq_lengths=seqs_train, padding=True, padding_value=0.0)
val_gen = val_generator(x_val, y_val)

In [26]:
epochs = 50
lr = 1e-4
optimizer = 'adam'
dropout = 0.2
hidden_1 = 64
hidden_2 = 32
counting_hidden_1 = 32
counting_dense_1 = 64
counting_dense_2 = 16

output_path = os.path.join('/scratch/sk7898/radar_counting/models/' + loss, model_type, datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
os.makedirs(output_path, exist_ok=True)

#Callbacks for the training
early_stopping = EarlyStopping(monitor="val_loss", patience=3, min_delta=1e-4, verbose=5, mode="auto")
reduce_LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)        
model_checkpoint = ModelCheckpoint(os.path.join(output_path, 'best_val_loss_model.h5'),\
                                   monitor="val_loss", verbose=5, save_best_only=True, mode="auto")
callbacks = [early_stopping, reduce_LR, model_checkpoint]
        
model = build_lstm_time_model(hidden_1, hidden_2, counting_hidden_1,\
                              counting_dense_1, counting_dense_2,\
                              learning_rate=lr, optimizer=optimizer, dropout=dropout)

H_train = model.fit_generator(train_gen, validation_data=val_gen, validation_steps=1,\
                              steps_per_epoch=n_bins, epochs=epochs, callbacks=callbacks)

Epoch 1/10
12/12 [==============================] - 87s 7s/step - loss: 1.5418 - mean_absolute_error: 1.6241 - val_loss: 1.0587 - val_mean_absolute_error: 0.3047
Epoch 2/10
12/12 [==============================] - 84s 7s/step - loss: 1.5355 - mean_absolute_error: 1.6212 - val_loss: 1.0571 - val_mean_absolute_error: 0.3010
Epoch 3/10
12/12 [==============================] - 85s 7s/step - loss: 1.5293 - mean_absolute_error: 1.6186 - val_loss: 1.0556 - val_mean_absolute_error: 0.2975
Epoch 4/10
12/12 [==============================] - 84s 7s/step - loss: 1.5230 - mean_absolute_error: 1.6157 - val_loss: 1.0541 - val_mean_absolute_error: 0.2939
Epoch 5/10
12/12 [==============================] - 86s 7s/step - loss: 1.5163 - mean_absolute_error: 1.6125 - val_loss: 1.0524 - val_mean_absolute_error: 0.2898
Epoch 6/10
12/12 [==============================] - 84s 7s/step - loss: 1.5085 - mean_absolute_error: 1.6090 - val_loss: 1.0502 - val_mean_absolute_error: 0.2844
Epoch 7/10
12/12 [==========

In [27]:
#predicted_train = model.predict_generator(train_gen, steps=int(len(seqs_train)/batch_size))
#print('Predicted Train: ', predicted_train)
test_gen = test_generator(x_test, y_test)
predicted_test = model.predict_generator(test_gen, steps=len(seqs_test))
print('Predicted Test: ', predicted_test)

Predicted Test:  [[0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.7635876 ]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0.76358783]
 [0

In [29]:
test_gen = val_generator(x_test, y_test)
H_evaluate = model.evaluate_generator(test_gen, steps=len(seqs_test))
print('Loss:', H_evaluate[0])
print('MAE:', H_evaluate[1])

Loss: 1.4069990118344624
MAE: 1.6218287870287895
